In [83]:
# Import libraries
import psycopg2
import psycopg2.extras
import time
import re
import os
import glob

from config import db as dbpwd

In [97]:
# Data base connection details
hostname = 'localhost'
database = 'real_estate_data_test'
username = 'postgres'
pwd = dbpwd
port_id = 5432
conn = None

# Load the input data (only the most recent file)
data_path = "../02_Merge_Data/data/*.csv"
listing = glob.glob(data_path)
latest_file = max(listing, key=os.path.getctime)

# Define the table name
regex = "(data_.+).csv"
match = re.findall(rf"{regex}", latest_file)
name = ''.join(match)
table_name = name.lower()
print(f'The input file will be saved to the db as a table named: {table_name}')

The input file will be saved to the db as a table named: data_houston_tx_houses_merged_2022_12_23_20_59


In [99]:
#connect to the database
conn = psycopg2.connect(host=hostname,
                       dbname=database,
                       user=username,
                       password=pwd,
                       port=port_id)  
#create a cursor object 
#cursor object is used to interact with the database
cur = conn.cursor()

#create table with same headers as csv file
cur.execute('DROP TABLE IF EXISTS {}'.format(table_name))
cur.execute('CREATE TABLE IF NOT EXISTS {}(Page text, Item text, ZID text, State text, City text, Number text, Street text, zipCode text, Lat text, Lng text, Price text, Image text, Bedrooms text, Bathrooms text, Status text, daysOnSale text)'.format(table_name))

# CHECK HOW TO CHANGE THE TYPE OF THE COLUMNS
#cur.execute('CREATE TABLE IF NOT EXISTS {}(Page int, Item int, ZID int, State text, City text, Number int, Street text, zipCode int, Lat float, Lng float, Price int, Image text, Bedrooms int, Bathrooms int, Status text, daysOnSale int)'.format(table_name))


#open the csv file using python standard file I/O
#copy file into the table just created 
with open(latest_file, 'r') as f:
    next(f) # Skip the header row.
    #f , <database name>, Comma-Seperated
    cur.copy_from(f, table_name, sep=',')
    #Commit Changes
    conn.commit()
    #Close connection
    conn.close()

f.close()